In [ ]:
from fuzzywuzzy import fuzz, process
import os
import pathlib as Path
from simple_cysh import *
#%matplotlib inline
os.chdir(r'Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\Eval Management\ACM_analysis')

In [ ]:
heat_map_folders = ["Pride Heat Maps_ Coaching Logs",
                    "Spirit Heat Maps_ Coaching Logs",
                    "Purpose Heat Maps_ Coaching Logs",
                    "Heat Maps _ Coaching Logs"]

# for x in os.walk("Z:\\Heat Maps _ Coaching Logs"):
#     print(x)
coach_xlsx_paths = eval(open('OC_paths.txt').read())

sheet_months = ['Oct','Nov','Dec','Jan','Feb','March','April','May']
sheet_months_clean = ['October 2017','November 2017','December 2017','January 2018','February 2018','March 2018','April 2018','May 2018']
sheet_months_dict = dict(zip(sheet_months, sheet_months_clean))

expected_cols = [('Month', ''),
                 ('ACM', ''),
                 ('        Effective Tutoring \xa0 \xa0 \xa0 \xa0 \xa0     \xa0(ELA and Math Strategies)','Rating'),
                 ('        Effective Tutoring \xa0 \xa0 \xa0 \xa0 \xa0     \xa0(ELA and Math Strategies)','Rubric Note'),
                 ('Application of Feedback and Training', 'Rating'),
                 ('Application of Feedback and Training', 'Rubric Note'),
                 ('Effective Student Engagement & Behavior Management', 'Rating'),
                 ('Effective Student Engagement & Behavior Management', 'Rubric Note'),
                 ('Lesson Planning and Preparation', 'Rating'),
                 ('Lesson Planning and Preparation', 'Rubric Note'),
                 ('Student Progress Monitoring', 'Rating'),
                 ('Student Progress Monitoring', 'Rubric Note')]

declare_cols = ['ACM',
                'Plan Rating',
                'Plan Rubric Note',
                'ET Rating',
                'ET Rubric Note',
                'ESE Rating',
                'ESE Rubric Note',
                'SPM Rating',
                'SPM Rubric Note',
                'Learn Rating',
                'Learn Rubric Note',
                'Month',
                'Folder']

In [ ]:
# Gather heatmap data
# want to flag which docs have headers out of order, or named different than expected
d = []
for filepath in coach_xlsx_paths:
    # use pd.ExcelFile instead of pd.read_excel in order to use variable sheetnames
    with pd.ExcelFile(filepath) as xl:
        for sheet in list(set(xl.sheet_names) & set(sheet_months)):
            # Returns ordered dict
            heatmap = pd.read_excel(xl, header=[2,3], 
                                    sheet_name=sheet, 
                                    usecols=14, index_col=None)
            heatmap = heatmap.reset_index()
            heatmap = heatmap.rename(columns={heatmap.columns.get_level_values(0).unique()[0]:'ACM'}, level=0)
            heatmap['Month'] = sheet
            missing_cols = set(expected_cols) - set(heatmap)
#             print(f"{Path.WindowsPath(filepath).parts[-2]}: {sheet}: {missing_cols}")
            # Run this line to rename cols only if they match our expected names
#             heatmap = heatmap[[column for column in list(heatmap) if column in expected_cols]]
            # Remove columns containing "Color"
            heatmap = heatmap[[column for column in list(heatmap) if str(column).find("Color") == -1]]
            heatmap['Folder'] = Path.WindowsPath(filepath).parts[-2]
            heatmap = heatmap[0:13]
            # Run this line to rename cols assuming universal position
            heatmap.columns = declare_cols
            d.append(heatmap)
            
raw_heatmaps_df = pd.concat(d, ignore_index=True)

heatmaps_df.to_csv('OC_raw.csv', index=False)

In [ ]:
# Load from offline copy
heatmaps_df = pd.read_csv('OC_raw.csv')
heatmaps_df = heatmaps_df.loc[~heatmaps_df['ACM'].isnull() &
                              (heatmaps_df['ACM']!='LLP Ratings') &
                              ~heatmaps_df['Plan Rating'].isnull() &
                              ~heatmaps_df['ET Rating'].isnull()]
heatmaps_df['Folder'] = heatmaps_df['Folder'].str.replace('Chicago ','')
heatmaps_df['Folder'] = heatmaps_df['Folder'].str.replace('Academy HS','Chicago Academy HS')
heatmaps_df['Month'].replace(sheet_months_dict, inplace=True)
heatmaps_df.loc[:, 'Month'] = pd.to_datetime(heatmaps_df['Month'])

# Load schools from salesforce
account_df = get_cysh_df('Account', ['Id', 'Name'])
account_df.rename(columns={'Id':'Organization__c', 'Name':'School'}, inplace=True)

d = []
for folder_name in set(heatmaps_df['Folder']):
    match = process.extract(folder_name, set(account_df['School']), scorer=fuzz.token_set_ratio, limit=1)
    match = [folder_name] + [x for tup in match for x in tup]
    d.append(match)
df = pd.DataFrame(d, columns=['Folder', 'School', 'Match_Score'])
df.head()

# merge
heatmaps_df = heatmaps_df.merge(df[['Folder', 'School']], how='left', on='Folder')
heatmaps_df['Key'] = heatmaps_df['ACM'] + heatmaps_df['School']

# Load staff from salesforce, merge in school name
staff_df = get_cysh_df('Staff__c', ['Id', 'Name', 'Site__c', 'Organization__c', 'Reference_Id__c', 'Role__c'])
acm_df = staff_df[(staff_df['Site__c']=="Chicago") & staff_df['Role__c'].str.contains('Corps Member')]
acm_df = acm_df.merge(account_df, how='left', on='Organization__c')

# iterate by team, search for matches only within team
d=[]
for school in set(heatmaps_df['School']):
    raw_acm_names = set(heatmaps_df['ACM'].loc[heatmaps_df['School']==school])
    legit_acm_names = set(acm_df['Name'].loc[acm_df['School']==school])
    for name in raw_acm_names:
        match = process.extract(name, legit_acm_names, limit=2)
        match = [name] + [x for tup in match for x in tup] + [school]
        d.append(match)
matches_df = pd.DataFrame(d, columns=['ACM_raw', 'ACM', 'Match1_Score', 'ACM2', 'Match2_Score', 'School'])
matches_df['Key'] = matches_df['ACM_raw'] + matches_df['School']

del heatmaps_df['ACM']

heatmaps_df = heatmaps_df.merge(matches_df[['ACM', 'Key']], how='left', on='Key')

del heatmaps_df['Key'], heatmaps_df['Folder']

In [ ]:
sch_ref_path = r"Z:\ChiPrivate\Chicago Data and Evaluation\SY18\SY18 Team Placement\SY18 FINAL Team Placements.xlsx"
sch_ref_df = pd.read_excel(sch_ref_path, sheet_name="School Data")
# Merge in Coach
heatmaps_df = heatmaps_df.merge(sch_ref_df[['School', 'Coach']], on='School')

# staff_df has 'ID__c' but it doesn't match to raw survey ID
staff_df = get_cysh_df('Staff__c', ['Id', 'Name'], "Site__c = 'Chicago'", rename_id=True)
# merge in ACM ID
heatmaps_df = heatmaps_df.merge(staff_df, left_on='ACM', right_on='Name', how='left')

In [ ]:
heatmaps_df = heatmaps_df[['Staff__c',
                           'ACM',
                           'School',
                           'Coach',
                           'Month',
                           'Plan Rating',
                           'Plan Rubric Note',
                           'ET Rating',
                           'ET Rubric Note',
                           'ESE Rating',
                           'ESE Rubric Note',
                           'SPM Rating',
                           'SPM Rubric Note',
                           'Learn Rating',
                           'Learn Rubric Note']]

# heatmaps_df.to_excel(r"Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\SY18 O&C Heat Map Rollup.xlsx",
#                      index=False, freeze_panes=(1,0))
heatmaps_df.to_csv('OC_clean.csv', index=False)

In [ ]:
heatmaps_df = pd.read_csv('OC_clean.csv')
heatmaps_df = heatmaps_df.loc[heatmaps_df.Month < '2018-01-01']

In [ ]:
cols = [column for column in heatmaps_df.columns if 'Rating' in column]

In [ ]:
# Normalize each Coaches' responses (for each observation, subtract mean and divide by SD)
for coach in heatmaps_df['Coach'].unique():
    print('')
    print(coach)
    for col in cols:
        descriptives = heatmaps_df.loc[heatmaps_df.Coach==coach, col].describe()
        print(col)
        print(f"mean: {descriptives['mean']}")
        print(f" std: {descriptives['mean']}")
        heatmaps_df.loc[heatmaps_df.Coach==coach, col+'_norm'] = (heatmaps_df.loc[heatmaps_df.Coach==coach, col] - descriptives['mean']) / descriptives['std']

In [ ]:
# Aggregate across months
heatmaps_agg = heatmaps_df.groupby('Staff__c').mean().reset_index()

In [ ]:
heatmaps_agg.to_csv('OC_clean_agg.csv', index=False)

## Next Steps
* Aggregate across categories?
* Clean other sheets? Not important, would only gain actual date of observation rather than month